<a href="https://colab.research.google.com/github/kdw9687/Machine-Learning/blob/main/making_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
from tqdm import tqdm
import numpy as np

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/KOPIS Data/[KOPIS]RawData(07.19)/2020년 1분기/test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (37,50,51,52) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df=pd.DataFrame(data)

In [ ]:
df.fillna(0,inplace=True)
df=df.drop_duplicates(['입장권고유번호'],keep='last')
df.drop(df.loc[df['예매/취소구분']==2].index, inplace=True)
df['예매/취소구분'].value_counts()

1    333251
Name: 예매/취소구분, dtype: int64

In [ ]:
unique_concert=df['공연코드'].unique()

In [ ]:
sex={}
for i in tqdm(range(len(df))):
  if sex.get(df.iloc[i]['공연코드'])==None:
    if df.iloc[i]['성별']==2:
      sex[df.iloc[i]['공연코드']]=[0,1,0]
    elif df.iloc[i]['성별']==1:
      sex[df.iloc[i]['공연코드']]=[1,0,0]
    else:
      sex[df.iloc[i]['공연코드']]=[0,0,1]
  else:
    if df.iloc[i]['성별']==2:
      sex[df.iloc[i]['공연코드']][1]=sex[df.iloc[i]['공연코드']][1]+1
    elif df.iloc[i]['성별']==1:
      sex[df.iloc[i]['공연코드']][0]=sex[df.iloc[i]['공연코드']][0]+1
    else:
      sex[df.iloc[i]['공연코드']][2]=sex[df.iloc[i]['공연코드']][2]+1

100%|██████████| 333251/333251 [04:57<00:00, 1121.55it/s]


In [ ]:
concert_info=pd.DataFrame(list(sex.values()),columns=['male','female','sex_none'],index=sex.keys())

In [ ]:
grade={}
for i in tqdm(range(len(df))):
  if grade.get(df.iloc[i]['공연코드'])==None:
    grade[df.iloc[i]['공연코드']]=df.iloc[i]['관람연령']

100%|██████████| 333251/333251 [01:05<00:00, 5113.08it/s]


In [ ]:
concert_grade=pd.DataFrame(list(grade.values()),columns=['grade'],index=grade.keys())

In [ ]:
df['연령']=df['연령'].fillna(1890)
year_range_sex={}
for i in tqdm(range(len(df))):
  if year_range_sex.get(df.iloc[i]['공연코드'])==None:
    year_range_sex[df.iloc[i]['공연코드']]={}
  sex=df.iloc[i]['성별']
  age=2020-df.iloc[i]['연령']
  age_range=age//10
  age_range=int(age_range)
  if sex==1:
    category='male'
  elif sex==2:
    category='female'
  else :
    category='Nothing'
  mini_cat=str(age_range)
  category=category+'_'+mini_cat
  if year_range_sex[df.iloc[i]['공연코드']].get(category)==None:
    year_range_sex[df.iloc[i]['공연코드']][category]=1
  else:
    year_range_sex[df.iloc[i]['공연코드']][category]+=1

100%|██████████| 333251/333251 [05:26<00:00, 1019.60it/s]


In [ ]:
for_columns=pd.DataFrame(list(year_range_sex.values()),index=year_range_sex.keys())

In [ ]:
for_columns

,female_3,Nothing_202,female_4,male_3,male_6,female_2,male_4,female_6,Nothing_3,male_7,Nothing_2,female_5,male_5,male_2,Nothing_4,male_202,female_7,Nothing_5,Nothing_1,female_1,male_1,female_8,Nothing_6,female_202,male_0,Nothing_10,Nothing_11,Nothing_7,Nothing_9,Nothing_8,male_8,male_10,female_0,female_12,Nothing_0,male_9,female_11,female_9,Nothing_12
PF377509,1965.0,1663.0,374.0,367.0,5.0,100.0,162.0,18.0,14.0,2.0,3.0,7.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PF375115,486.0,42.0,310.0,58.0,NaN,10.0,62.0,2.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PF378830,1896.0,764.0,304.0,274.0,7.0,66.0,115.0,2.0,14.0,2.0,4.0,10.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PF378785,1897.0,1831.0,526.0,322.0,3.0,60.0,179.0,8.0,100.0,NaN,6.0,7.0,7.0,NaN,32.0,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PF375872,384.0,28.0,107.0,85.0,NaN,9.0,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PF371900,7.0,NaN,4.0,2.0,NaN,11.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PF371741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PF371024,NaN,NaN,NaN,15.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PF382519,5.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
concert_data_sexrange=pd.DataFrame(list(year_range_sex.values()),index=year_range_sex.keys(),columns=sorted(for_columns.columns))
concert_data_sexrange.fillna(0,inplace=True)

In [ ]:
genre={}
for i in tqdm(range(len(df))):
  if genre.get(df.iloc[i]['공연코드'])==None:
    genre[df.iloc[i]['공연코드']]=df.iloc[i]['장르명']


100%|██████████| 333251/333251 [01:05<00:00, 5112.97it/s]


In [ ]:
concert_genre=pd.DataFrame(list(genre.values()),index=list(genre.keys()),columns=['genre'])

In [ ]:
location={}
for i in tqdm(range(len(df))):
  if location.get(df.iloc[i]['공연코드'])==None:
    location[df.iloc[i]['공연코드']]=[]
    location[df.iloc[i]['공연코드']].append(df.iloc[i]['공연지역명'])
  elif df.iloc[i]['공연지역명'] not in location[df.iloc[i]['공연코드']]:
    location[df.iloc[i]['공연코드']].append(df.iloc[i]['공연지역명'])

100%|██████████| 333251/333251 [03:12<00:00, 1727.17it/s]


In [ ]:
concert_location=pd.DataFrame(list(location.values()),index=list(location.keys()),columns=['location1','location2'])

In [ ]:
start_date={}
for i in tqdm(range(len(df))):
  if start_date.get(df.iloc[i]['공연코드'])==None:
    start_date[df.iloc[i]['공연코드']]=[]
    start_date[df.iloc[i]['공연코드']].append(df.iloc[i]['공연시작일자'])
  elif df.iloc[i]['공연시작일자'] not in start_date[df.iloc[i]['공연코드']]:
    start_date[df.iloc[i]['공연코드']].append(df.iloc[i]['공연시작일자'])

100%|██████████| 333251/333251 [03:12<00:00, 1728.89it/s]


In [ ]:
end_date={}
for i in tqdm(range(len(df))):
  if end_date.get(df.iloc[i]['공연코드'])==None:
    end_date[df.iloc[i]['공연코드']]=[]
    end_date[df.iloc[i]['공연코드']].append(df.iloc[i]['공연종료일자'])
  elif df.iloc[i]['공연종료일자'] not in end_date[df.iloc[i]['공연코드']]:
    end_date[df.iloc[i]['공연코드']].append(df.iloc[i]['공연종료일자'])

100%|██████████| 333251/333251 [03:13<00:00, 1719.01it/s]


In [ ]:
df_start_date=pd.DataFrame(list(start_date.values()),index=start_date.keys(),columns=['start_date'])
df_end_date=pd.DataFrame(list(end_date.values()),index=end_date.keys(),columns=['end_date'])
concert_date=pd.concat([df_start_date,df_end_date],axis=1)

In [ ]:
award={}
for i in tqdm(range(len(df))):
  if award.get(df.iloc[i]['공연코드'])==None:
    award[df.iloc[i]['공연코드']]=[]
    award[df.iloc[i]['공연코드']].append(df.iloc[i]['수상실적'])
  elif df.iloc[i]['수상실적'] not in award[df.iloc[i]['공연코드']]:
    award[df.iloc[i]['공연코드']].append(df.iloc[i]['수상실적'])

 85%|████████▌ | 284370/333251 [02:44<00:28, 1724.58it/s]

In [ ]:
concert_award=pd.DataFrame(list(award.values()),index=award.keys(),columns=['award'])

In [ ]:
price={}
for i in tqdm(range(len(df))):
  if price.get(df.iloc[i]['공연코드'])==None:
    price[df.iloc[i]['공연코드']]=[]
    price[df.iloc[i]['공연코드']].append(df.iloc[i]['좌석등급'])
  elif df.iloc[i]['좌석등급'] not in price[df.iloc[i]['공연코드']]:
    price[df.iloc[i]['공연코드']].append(df.iloc[i]['좌석등급'])

 93%|█████████▎| 311350/333251 [02:56<00:12, 1794.60it/s]

In [ ]:
concert_price=pd.DataFrame(list(price.values()),index=price.keys(),columns=['price'])

In [ ]:
time={}
for i in tqdm(range(len(df))):
  if time.get(df.iloc[i]['공연코드'])==None:
    time[df.iloc[i]['공연코드']]=[]
    time[df.iloc[i]['공연코드']].append(df.iloc[i]['소요시간'])
  elif df.iloc[i]['소요시간'] not in time[df.iloc[i]['공연코드']]:
    time[df.iloc[i]['공연코드']].append(df.iloc[i]['소요시간'])

100%|██████████| 333251/333251 [03:09<00:00, 1762.16it/s]


In [ ]:
concert_time=pd.DataFrame(list(time.values()),index=time.keys(),columns=['time'])

In [ ]:
concert_time.fillna(0,inplace=True)
for i in range(len(concert_time)):
  minute=0
  tmp=concert_time.iloc[i][0]
  if tmp==0:
    continue
  if '1시간' in concert_time.iloc[i][0]:
    minute+=60
    tmp=tmp.replace('1시간','')
  elif '2시간' in concert_time.iloc[i][0]:
    minute+=120
    tmp=tmp.replace('2시간','')
  elif '3시간' in concert_time.iloc[i][0]:
    minute+=180
    tmp=tmp.replace('3시간','')
  if '분' in concert_time.iloc[i][0]:
    tmp=tmp.replace('분','')
    tmp=tmp.strip()
    minute+=int(tmp)
  concert_time.iloc[i][0]=minute

In [ ]:
maxim=[]
minim=[]
for i in tqdm(range(len(concert_price))):
  str=concert_price.iloc[i][0]
  items = re.findall('\(([^)]\d+)', str)
  if '전석무료' in str:
    maxim.append(0)
    minim.append(0)
    continue
  spec_price=[]
  for value in items :
    spec_price.append(int(value))
  maxi=max(spec_price)
  mini=min(spec_price)
  maxim.append(maxi)
  minim.append(mini)

concert_price['max_price']=maxim
concert_price['min_price']=minim

100%|██████████| 366/366 [00:00<00:00, 10422.12it/s]


In [ ]:
seat={}
for i in tqdm(range(len(df))):
  if seat.get(df.iloc[i]['공연코드'])==None:
    seat[df.iloc[i]['공연코드']]=[]
    seat[df.iloc[i]['공연코드']].append(df.iloc[i]['좌석수'])
  elif df.iloc[i]['좌석수'] not in seat[df.iloc[i]['공연코드']]:
    seat[df.iloc[i]['공연코드']].append(df.iloc[i]['좌석수'])

100%|██████████| 333251/333251 [03:16<00:00, 1691.66it/s]


In [ ]:
concert_seat=pd.DataFrame(list(seat.values()),index=seat.keys(),columns=['seat1','seat2'])

In [ ]:
festival={}
for i in tqdm(range(len(df))):
  if festival.get(df.iloc[i]['공연코드'])==None:
    festival[df.iloc[i]['공연코드']]=[]
    if df.iloc[i]['축제 여부']=='Y':
      festival[df.iloc[i]['공연코드']].append(1)
    else:
      festival[df.iloc[i]['공연코드']].append(0)

100%|██████████| 333251/333251 [01:10<00:00, 4715.52it/s]


In [ ]:
concert_festival=pd.DataFrame(list(festival.values()),index=festival.keys(),columns=['festival'])

In [ ]:
buildings={}
for i in tqdm(range(len(df))):
  if buildings.get(df.iloc[i]['공연코드'])==None:
    buildings[df.iloc[i]['공연코드']]=[]
    buildings[df.iloc[i]['공연코드']].append(df.iloc[i]['시설특성'])
  elif df.iloc[i]['시설특성'] not in buildings[df.iloc[i]['공연코드']]:
    buildings[df.iloc[i]['공연코드']].append(df.iloc[i]['시설특성'])

100%|██████████| 333251/333251 [03:17<00:00, 1686.14it/s]


In [ ]:
concert_buildings=pd.DataFrame(list(buildings.values()),index=buildings.keys(),columns=['buildings1','buildings2'])

In [ ]:
openrun={}
for i in tqdm(range(len(df))):
  if openrun.get(df.iloc[i]['공연코드'])==None:
    openrun[df.iloc[i]['공연코드']]=[]
    if df.iloc[i]['오픈런 여부']=='Y':
      openrun[df.iloc[i]['공연코드']].append(1)
    else:
      openrun[df.iloc[i]['공연코드']].append(0)

100%|██████████| 333251/333251 [01:08<00:00, 4877.29it/s]


In [ ]:
concert_openrun=pd.DataFrame(list(openrun.values()),index=openrun.keys(),columns=['openrun'])

In [ ]:
foreign={}
for i in tqdm(range(len(df))):
  if foreign.get(df.iloc[i]['공연코드'])==None:
    foreign[df.iloc[i]['공연코드']]=[]
    if df.iloc[i]['내한공연 여부']=='Y':
      foreign[df.iloc[i]['공연코드']].append(1)
    else:
      foreign[df.iloc[i]['공연코드']].append(0)

100%|██████████| 333251/333251 [01:09<00:00, 4823.39it/s]


In [ ]:
concert_foreign=pd.DataFrame(list(foreign.values()),index=foreign.keys(),columns=['foreign'])

In [ ]:
result=pd.concat([concert_genre,concert_grade,concert_time,concert_seat,concert_location,concert_buildings,concert_award,concert_price,concert_date,concert_info,concert_data_sexrange,concert_festival,concert_foreign,concert_openrun],axis=1)

In [ ]:
result

,genre,grade,time,seat1,seat2,location1,location2,buildings1,buildings2,award,price,max_price,min_price,start_date,end_date,male,female,sex_none,Nothing_0,Nothing_1,Nothing_10,Nothing_11,Nothing_12,Nothing_2,Nothing_202,Nothing_3,Nothing_4,Nothing_5,Nothing_6,Nothing_7,Nothing_8,Nothing_9,female_0,female_1,female_11,female_12,female_2,female_202,female_3,female_4,female_5,female_6,female_7,female_8,female_9,male_0,male_1,male_10,male_2,male_202,male_3,male_4,male_5,male_6,male_7,male_8,male_9,festival,foreign,openrun
PF377509,뮤지컬,전체 관람가,70,998,NaN,서울,None,민간(대학로 외),None,0,"R석(평일)(50000), R석(주말)(60000)",60000,50000,2019-12-07,2020-01-19,542,2464,1683,0.0,0.0,0.0,0.0,0.0,3.0,1663.0,14.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,1965.0,374.0,7.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,367.0,162.0,3.0,5.0,2.0,0.0,0.0,0,0,0
PF375115,뮤지컬,24개월 이상,60,134,NaN,서울,None,민간(대학로),None,"2005 아시테지 서울 어린이 연극상 연극상, 2005 아시테지 서울 어린이 연극상...",전석(30000),30000,30000,2019-10-31,2021-01-23,121,818,42,0.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,486.0,310.0,10.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,58.0,62.0,0.0,0.0,0.0,0.0,0.0,0,0,1
PF378830,뮤지컬,전체 관람가,70,450,NaN,서울,None,민간(대학로 외),None,0,"평일R석(50000), 주말R석(60000)",60000,50000,2019-12-25,2020-02-02,403,2278,782,0.0,0.0,0.0,0.0,0.0,4.0,764.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,0.0,1896.0,304.0,10.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,274.0,115.0,0.0,7.0,2.0,0.0,0.0,0,0,0
PF378785,뮤지컬,24개월 이상,65,601,NaN,서울,None,공공(문예회관),None,0,"R석(66000), S석(44000)",66000,44000,2019-12-13,2020-02-02,511,2501,1971,0.0,0.0,0.0,0.0,0.0,6.0,1831.0,100.0,32.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,1897.0,526.0,7.0,8.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,322.0,179.0,7.0,3.0,0.0,0.0,0.0,0,0,0
PF375872,뮤지컬,24개월 이상,55,200,NaN,서울,None,민간(대학로 외),None,0,전석(40000),40000,40000,2019-10-02,2020-01-05,147,500,28,0.0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,384.0,107.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,62.0,0.0,0.0,0.0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PF371900,연극,만 13세 이상,100,100,NaN,서울,None,민간(대학로 외),None,0,전석(30000),30000,30000,2020-01-10,2020-01-19,5,22,2,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,7.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
PF371741,연극,만 12세 이상,90,200,NaN,서울,None,민간(대학로),None,0,전석(35000),35000,35000,2020-01-10,2020-02-09,0,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
PF371024,클래식,만 7세 이상,100,390,NaN,서울,None,민간(대학로 외),None,0,전석(20000),20000,20000,2020-01-10,2020-01-10,18,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,15.0,0.0,0.0,2.0,0.0,0.0,0.0,0,0,0
PF382519,무용,만 15세 이상,50,30,NaN,서울,None,공공(기타),None,0,전석(20000),20000,20000,2020-01-10,2020-01-12,1,6,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [ ]:
result.to_csv('real_result_end.csv',encoding='utf-8',mode='a')